# Sentence Splitter: Out of Domain Evaluation (Generative models)

In [ ]:
!pip install --upgrade pip
!pip install transformers evaluate torch numpy pandas datasets jupyter unsloth

Import the libraries

In [ ]:
import random
import numpy as np
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer

Before proceeding, make the run as deterministic as possible:

In [ ]:
RANDOM_STATE = 777

def set_seed(seed=777, total_determinism=False):
    seed = seed
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if total_determinism:
        torch.use_deterministic_algorithms(True)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
set_seed(RANDOM_STATE) # Set the seed for reproducibility -- use_deterministic_algorithms can make training slower :(

Reuse the `Prompt` class from the training notebook.
Here only the `question` method will be invoked.

In [ ]:
class Prompt:
    def __init__(self, input_text):
        self.input_text = input_text

    def instruction(self):
        return f"""Dividi il seguente testo italiano in frasi. Per favore rispondi con una frase per riga. Grazie.

Testo: {self.input_text}
"""

    def conversation(self, output_text):
        return[
            {"role" : "system",    "content" : "Sei un esperto di linguistica italiana specializzato nella segmentazione delle frasi."},
            {"role" : "user",      "content" : self.instruction()},
            {"role" : "assistant", "content" : output_text},
        ]

    def question(self):
        return[
            {"role" : "system",    "content" : "Sei un esperto di linguistica italiana specializzato nella segmentazione delle frasi."},
            {"role" : "user",      "content" : self.instruction()},
        ]

Let's test (this time manually) how the trained LLM-based models perform on a given text
using an out of domain novel (Cuore).

In [ ]:
input_text = """Non era un legno di lusso, ma un semplice pezzo
da catasta, di quelli che d’inverno si mettono nelle
stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze.
Non so come andasse, ma il fatto gli è che un bel
giorno questo pezzo di legno capitò nella bottega
di un vecchio falegname, il quale aveva nome mastr’Antonio, se non che tutti lo chiamavano maestro
Ciliegia, per via della punta del suo naso, che era
sempre lustra e paonazza, come una ciliegia matura.
Appena maestro Ciliegia ebbe visto quel pezzo di
legno, si rallegrò tutto; e dandosi una fregatina di
mani per la contentezza, borbottò a mezza voce:
"Questo legno è capitato a tempo; voglio servirmene per fare una gamba di tavolino." 
"""
input_text = input_text.splitlines()
input_text = " ".join(input_text)
input_text

Each model is loaded from Hugging Face and used to generate the answer.

In [ ]:
def test_model(name):
    model, tokenizer = FastLanguageModel.from_pretrained(
        'fax4ever/' + name, 
        load_in_4bit=True, 
        dtype=None, 
        max_seq_length=512
    )
    model = FastLanguageModel.for_inference(model)
    question = tokenizer.apply_chat_template(
        [Prompt(input_text).question()], 
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
        enable_thinking = False, # Disable thinking
    )

    model.generate(
        **tokenizer(question, return_tensors = "pt").to("cuda"),
        max_new_tokens = 512,
        temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

Testing one model at a time (commenting the others) to avoid memory issues,
restarting the Kernel to free the GPU memory!

In [ ]:
# test_model("qwen3-4b-unsloth-bnb-4bit-sentence-splitter")

In [ ]:
# test_model("meta-llama-3.1-8b-instruct-unsloth-bnb-4bit-sentence-splitter")

In [ ]:
# test_model("mistral-7b-instruct-v0.3-bnb-4bit-sentence-splitter")

In [ ]:
test_model("Minerva-7B-instruct-v1.0-sentence-splitter")